In [0]:
import IPython.display as ipd
!rm -rf *
!wget https://github.com/PvtSec/weatherML/raw/master/agro_model.zip -O model.zip
!wget https://github.com/PvtSec/weatherML/raw/84eedf65cd5a4e2e089464a50e0b4c01d06fc719/Dataset/Delhi-Hourly/clean2.csv -O weather_data.csv
!unzip model.zip
!rm model.zip
!clear
ipd.clear_output()

In [0]:
import tensorflow as tf
from numpy import array

In [0]:
lstm = tf.keras.models.load_model('agro_model')

In [0]:
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import numpy as np



#set output graph diamension
rcParams['figure.figsize'] = 10, 6

data = pd.read_csv("weather_data.csv")
temp_data_init = data['temperature']
temp_data_init.index = data['Index']

split = 100000
temp_data = temp_data_init.values
temp_mean = temp_data[:split].mean()
temp_stdv = temp_data[:split].std()
temp_std = (temp_data - temp_mean) / temp_stdv


past_data = 99000
target_data = 0

data = []
labels = []

start_index = 0 + past_data
end_index = split

for i in range(start_index, end_index):
  index = range(i-past_data, i)
  data.append(np.reshape(temp_std[index], (past_data, 1)))
  labels.append(temp_std[i+target_data])
x,y = np.array(data), np.array(labels)

train_univariate = tf.data.Dataset.from_tensor_slices((x, y))
train_univariate = train_univariate.cache().shuffle(256).batch(450).repeat()
val_univariate = tf.data.Dataset.from_tensor_slices((x, y))
val_univariate = val_univariate.batch(256).repeat()

In [0]:
for xx, yy in val_univariate.take(1):
  plot_data = [xx[0].numpy(), yy[0].numpy(), lstm.predict(xx)[0]]

In [0]:
std_real_future = plot_data[1]
std_pred_future = plot_data[2][0]

#convert standardized value back to temperature
actual = abs((std_real_future * temp_stdv)+temp_mean)
predicted = abs((std_pred_future * temp_stdv)+temp_mean)
error = abs(round(((actual - predicted) / actual)*100,2))
print("Actual    :",round(actual,2),"°C")
print("Predicted :",round(predicted,2),"°C")
print("Error     :",error,"%")